### **03 Worldwide Earthquake Events API - Gold Layer Processing **

In [18]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 22, Finished, Available, Finished)

In [19]:
#rough
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)


StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 23, Finished, Available, Finished)

NameError: name 'start_date' is not defined

In [5]:
coordinates = (25.425,142.6876)
rg.search(coordinates)

StatementMeta(, dba8259b-adbb-4b3d-8ae7-4e6a3cb4e38a, 9, Finished, Available, Finished)

Loading formatted geocoded file...


[{'lat': '34.98333',
  'lon': '139.86667',
  'name': 'Tateyama',
  'admin1': 'Chiba',
  'admin2': '',
  'cc': 'JP'}]

In [4]:
#rough
display(df)

StatementMeta(, dba8259b-adbb-4b3d-8ae7-4e6a3cb4e38a, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8439bd5b-add7-4d42-928f-2a55fed610e4)

In [9]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 13, Finished, Available, Finished)

In [10]:

# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 14, Finished, Available, Finished)

In [11]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 15, Finished, Available, Finished)

In [24]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 28, Finished, Available, Finished)

In [ ]:
#display(df_with_location_sig_class)

In [31]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')
     

StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 35, Finished, Available, Finished)

In [30]:
#df_with_location_sig_class.write.format("delta").mode("overwrite").save("abfss://757214c9-ee6d-4353-a345-4bf728a48f27@onelake.dfs.fabric.microsoft.com/65653cc4-cbae-45b0-8aca-d63ab503f142/Tables/dbo/earthquake_events_gold")


StatementMeta(, a60c5760-90cf-4eb7-ab0b-7d161d4a789d, 34, Finished, Available, Finished)